# TCN Classification with MR Dataset
<hr>

We will build a text classification model using TCN model on the Movie Review Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [5]:
corpus = pd.read_pickle('../../../0_data/TREC/TREC.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(5952, 3)


,sentence,label,split
0,how did serfdom develop in and then leave russ...,0,train
1,what films featured the character popeye doyle ?,1,train
2,how can i find a list of celebrities ' real na...,0,train
3,what fowl grabs the spotlight after the chines...,1,train
4,what is the full form of .com ?,2,train
...,...,...,...
5947,who was the 22nd president of the us ?,3,test
5948,what is the money they use in zambia ?,1,test
5949,how many feet in a mile ?,5,test
5950,what is the birthstone of october ?,1,test


In [6]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5952 non-null   object
 1   label     5952 non-null   int32 
 2   split     5952 non-null   object
dtypes: int32(1), object(2)
memory usage: 116.4+ KB


In [8]:
corpus.groupby(by=['split', 'label']).count()

sentence
split label          
test  0           138
      1            94
      2             9
      3            65
      4            81
      5           113
train 0          1162
      1          1250
      2            86
      3          1223
      4           835
      5           896

In [9]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

5452
5452
500
500


In [10]:
train_x[0]

'how did serfdom develop in and then leave russia ?'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [11]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  what is the full form of .com ?
Into a sequence of int: [3, 4, 2, 471, 261, 5, 372]
Into a padded sequence: [  3   4   2 471 261   5 372   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [13]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
what 3
is 4
of 5
in 6
a 7
how 8
's 9
was 10
8461


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [14]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(6, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [15]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [13]:
# tcn_full_summary(model_0)

In [16]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [17]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the input shape
        model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.7):

            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            else:
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 24s 139ms/step - loss: 1.5604 - accuracy: 0.3325 - val_loss: 0.8038 - val_accuracy: 0.7200
Epoch 2/100
110/110 [==============================] - 10s 94ms/step - loss: 0.6618 - accuracy: 0.7846 - val_loss: 0.5389 - val_accuracy: 0.8120
Epoch 3/100
110/110 [==============================] - 10s 93ms/step - loss: 0.3117 - accuracy: 0.9115 - val_loss: 0.4700 - val_accuracy: 0.8760
Epoch 4/100
110/110 [==============================] - 10s 94ms/step - loss: 0.1802 - accuracy: 0.9460 - val_loss: 0.5101 - val_accuracy: 0.8640
Epoch 5/100
110/110 [==============================] - 10s 95ms/step - loss: 0.1116 - accuracy: 0.9617 - val_loss: 0.5702 - val_accuracy: 0.8620 0.1115 - accuracy
Epoch 6/100
110/110 [==============================] - 10s 95ms/step - loss: 0.0821 - accuracy: 0.9690 - val_loss: 0.5980 - 

110/110 [==============================] - 10s 91ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 1.0137 - val_accuracy: 0.8840
Epoch 27/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0047 - accuracy: 0.9983 - val_loss: 1.0557 - val_accuracy: 0.8840
Epoch 28/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 1.1287 - val_accuracy: 0.8860
Epoch 29/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0015 - accuracy: 0.9994 - val_loss: 1.1974 - val_accuracy: 0.8800
Epoch 30/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 0.9813 - val_accuracy: 0.8600
Epoch 31/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0270 - accuracy: 0.9926 - val_loss: 0.7349 - val_accuracy: 0.8780
Epoch 32/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0331 - accuracy: 0.9947 - val_loss: 1.2165 - val_accuracy

Epoch 35/100
110/110 [==============================] - 10s 92ms/step - loss: 0.0196 - accuracy: 0.9959 - val_loss: 0.6489 - val_accuracy: 0.8680
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
Test Accuracy: 89.99999761581421
Done!

  Activation Filters    Acc
0       relu       1  0.882
1       relu       2  0.888
2       relu       3  0.900

-------------------------------------------
Training 4: relu activation, 4 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 15s 98ms/step - loss: 1.6547 - accuracy: 0.2595 - val_loss: 1.1950 - val_accuracy: 0.4580
Epoch 2/100
110/110 [==============================] - 10s 91ms/step - loss: 1.0219 - accuracy: 0.5726 - val_loss: 0.5496 - val_accuracy: 0.8540
Epoch 3/100
110/110 [==============================] - 10s 91ms/step - loss: 0.4838 - accuracy: 0.8317 - val_loss: 0.5225 - val_accuracy: 0.8520
Epoch 4/100
110/110 [=============================

Epoch 19/100
110/110 [==============================] - 10s 95ms/step - loss: 0.0148 - accuracy: 0.9955 - val_loss: 0.9825 - val_accuracy: 0.8800
Epoch 20/100
110/110 [==============================] - 11s 104ms/step - loss: 0.0052 - accuracy: 0.9983 - val_loss: 1.2073 - val_accuracy: 0.8880
Epoch 21/100
110/110 [==============================] - 11s 102ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 1.0428 - val_accuracy: 0.8640
Epoch 22/100
110/110 [==============================] - 11s 100ms/step - loss: 0.0241 - accuracy: 0.9949 - val_loss: 0.8475 - val_accuracy: 0.8620
Epoch 23/100
110/110 [==============================] - 13s 114ms/step - loss: 0.0523 - accuracy: 0.9888 - val_loss: 0.8480 - val_accuracy: 0.8720
Epoch 24/100
110/110 [==============================] - 14s 126ms/step - loss: 0.0169 - accuracy: 0.9963 - val_loss: 0.9073 - val_accuracy: 0.8760
Epoch 25/100
110/110 [==============================] - 13s 114ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 1.

Epoch 25/100
110/110 [==============================] - 11s 99ms/step - loss: 0.0625 - accuracy: 0.9786 - val_loss: 1.0736 - val_accuracy: 0.8760
Epoch 26/100
110/110 [==============================] - 11s 101ms/step - loss: 0.0601 - accuracy: 0.9848 - val_loss: 1.7107 - val_accuracy: 0.8640
Epoch 27/100
110/110 [==============================] - 11s 98ms/step - loss: 0.0512 - accuracy: 0.9835 - val_loss: 1.5590 - val_accuracy: 0.8140
Epoch 28/100
110/110 [==============================] - 11s 97ms/step - loss: 0.0584 - accuracy: 0.9815 - val_loss: 2.7029 - val_accuracy: 0.8520
Epoch 29/100
110/110 [==============================] - 11s 97ms/step - loss: 0.1011 - accuracy: 0.9802 - val_loss: 1.5800 - val_accuracy: 0.8420
Epoch 30/100
110/110 [==============================] - 12s 105ms/step - loss: 0.0830 - accuracy: 0.9794 - val_loss: 0.9759 - val_accuracy: 0.8760
Epoch 31/100
110/110 [==============================] - 11s 99ms/step - loss: 0.0751 - accuracy: 0.9780 - val_loss: 1.1136

Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Test Accuracy: 87.99999952316284
Done!

  Activation Filters    Acc
0       relu       1  0.882
1       relu       2  0.888
2       relu       3  0.900
3       relu       4  0.894
4       relu       5  0.890
5       relu       6  0.886
6       tanh       1  0.880

-------------------------------------------
Training 8: tanh activation, 2 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 21s 146ms/step - loss: 1.3329 - accuracy: 0.4709 - val_loss: 0.5913 - val_accuracy: 0.8260
Epoch 2/100
110/110 [==============================] - 12s 108ms/step - loss: 0.5408 - accuracy: 0.8256 - val_loss: 0.4436 - val_accuracy: 0.8460
Epoch 3/100
110/110 [==============================] - 13s 117ms/step - loss: 0.2262 - accuracy: 0.9379 - val_loss: 0.4302 - val_accuracy: 0.8380
Epoch 4/100
110/110 [==============================] - 15s 133ms/step - loss: 0.1

110/110 [==============================] - 10s 92ms/step - loss: 0.1918 - accuracy: 0.9543 - val_loss: 0.7074 - val_accuracy: 0.8220
Epoch 5/100
110/110 [==============================] - 10s 92ms/step - loss: 0.0885 - accuracy: 0.9749 - val_loss: 0.5315 - val_accuracy: 0.8760
Epoch 6/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0931 - accuracy: 0.9755 - val_loss: 0.7374 - val_accuracy: 0.8540
Epoch 7/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0571 - accuracy: 0.9879 - val_loss: 0.7664 - val_accuracy: 0.8440
Epoch 8/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0455 - accuracy: 0.9877 - val_loss: 0.7312 - val_accuracy: 0.8660
Epoch 9/100
110/110 [==============================] - 10s 93ms/step - loss: 0.0472 - accuracy: 0.9869 - val_loss: 0.8911 - val_accuracy: 0.8560
Epoch 10/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0403 - accuracy: 0.9885 - val_loss: 0.9226 - val_accuracy: 0.8

110/110 [==============================] - 10s 91ms/step - loss: 0.0323 - accuracy: 0.9930 - val_loss: 0.9627 - val_accuracy: 0.8380
Epoch 22/100
110/110 [==============================] - 10s 92ms/step - loss: 0.0348 - accuracy: 0.9914 - val_loss: 0.7701 - val_accuracy: 0.8720
Epoch 23/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0224 - accuracy: 0.9939 - val_loss: 0.8848 - val_accuracy: 0.8720
Epoch 24/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0098 - accuracy: 0.9981 - val_loss: 1.0880 - val_accuracy: 0.8420
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Test Accuracy: 89.80000019073486
Done!

  Activation Filters    Acc
0       relu       1  0.882
1       relu       2  0.888
2       relu       3  0.900
3       relu       4  0.894
4       relu       5  0.890
5       relu       6  0.886
6       tanh       1  0.880
7       tanh       2  0.892
8       tanh       3  0.880
9       tanh       4  0.8

110/110 [==============================] - 10s 90ms/step - loss: 0.0673 - accuracy: 0.9802 - val_loss: 0.8786 - val_accuracy: 0.8300
Epoch 14/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0274 - accuracy: 0.9907 - val_loss: 0.8340 - val_accuracy: 0.8660
Epoch 15/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0155 - accuracy: 0.9951 - val_loss: 1.0325 - val_accuracy: 0.8220
Epoch 16/100
110/110 [==============================] - 10s 90ms/step - loss: 0.0110 - accuracy: 0.9972 - val_loss: 0.8398 - val_accuracy: 0.8580
Epoch 17/100
110/110 [==============================] - 10s 91ms/step - loss: 0.0175 - accuracy: 0.9958 - val_loss: 0.9733 - val_accuracy: 0.8380
Epoch 18/100
110/110 [==============================] - 10s 92ms/step - loss: 0.0132 - accuracy: 0.9959 - val_loss: 1.0710 - val_accuracy: 0.8300
Epoch 19/100
110/110 [==============================] - 10s 92ms/step - loss: 0.0180 - accuracy: 0.9958 - val_loss: 1.1054 - val_accuracy

## Summary

In [19]:
record.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
2,relu,3,0.900
9,tanh,4,0.898
3,relu,4,0.894
7,tanh,2,0.892
4,relu,5,0.890
1,relu,2,0.888
5,relu,6,0.886
0,relu,1,0.882
6,tanh,1,0.880
8,tanh,3,0.880


In [20]:
report = record.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [21]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [22]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [23]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [24]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 7526 words present from 8761 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [25]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [26]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [27]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## TCN Model

In [28]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(6, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [29]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 100, 300)     300000      input_14[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_13 (SpatialDr (None, 100, 300)     0           embedding_13[0][0]               
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_13[0][0]       
___________________________________________________________________________________________

## Train and Test the Model

In [30]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [31]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record2 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):

            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 8s 51ms/step - loss: 1.6586 - accuracy: 0.2539 - val_loss: 1.1601 - val_accuracy: 0.5320
Epoch 2/100
110/110 [==============================] - 7s 61ms/step - loss: 1.0465 - accuracy: 0.5740 - val_loss: 0.7761 - val_accuracy: 0.7300
Epoch 3/100
110/110 [==============================] - 7s 63ms/step - loss: 0.7100 - accuracy: 0.7412 - val_loss: 0.6517 - val_accuracy: 0.8280
Epoch 4/100
110/110 [==============================] - 7s 63ms/step - loss: 0.5298 - accuracy: 0.8058 - val_loss: 0.4062 - val_accuracy: 0.8560
Epoch 5/100
110/110 [==============================] - 6s 58ms/step - loss: 0.3496 - accuracy: 0.8829 - val_loss: 0.3442 - val_accuracy: 0.8820
Epoch 6/100
110/110 [==============================] - 6s 57ms/step - loss: 0.2555 - accuracy: 0.9135 - val_loss: 0.4897 - val_accuracy: 0.8440
Epoc

110/110 [==============================] - 7s 61ms/step - loss: 0.0225 - accuracy: 0.9943 - val_loss: 0.6490 - val_accuracy: 0.8920
Epoch 57/100
110/110 [==============================] - 7s 61ms/step - loss: 0.0297 - accuracy: 0.9938 - val_loss: 0.6143 - val_accuracy: 0.9140
Epoch 58/100
110/110 [==============================] - 7s 63ms/step - loss: 0.0092 - accuracy: 0.9981 - val_loss: 0.6116 - val_accuracy: 0.9080
Epoch 59/100
110/110 [==============================] - 7s 60ms/step - loss: 0.0192 - accuracy: 0.9944 - val_loss: 0.7037 - val_accuracy: 0.9160
Epoch 60/100
110/110 [==============================] - 7s 61ms/step - loss: 0.0246 - accuracy: 0.9947 - val_loss: 0.9179 - val_accuracy: 0.8880
Epoch 61/100
110/110 [==============================] - 7s 60ms/step - loss: 0.0215 - accuracy: 0.9944 - val_loss: 0.6727 - val_accuracy: 0.8820
Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping
Test Accuracy: 93.59999895095825
Done!

  Activation Filter

110/110 [==============================] - 7s 60ms/step - loss: 0.0615 - accuracy: 0.9788 - val_loss: 0.8444 - val_accuracy: 0.8240
Epoch 50/100
110/110 [==============================] - 6s 59ms/step - loss: 0.0767 - accuracy: 0.9752 - val_loss: 0.7402 - val_accuracy: 0.8820
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
Test Accuracy: 91.00000262260437
Done!

  Activation Filters    Acc
0       relu       1  0.936
1       relu       2  0.910

-------------------------------------------
Training 3: relu activation, 3 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 18s 66ms/step - loss: 1.7882 - accuracy: 0.2228 - val_loss: 1.4385 - val_accuracy: 0.4820
Epoch 2/100
110/110 [==============================] - 6s 58ms/step - loss: 1.4610 - accuracy: 0.3582 - val_loss: 1.2057 - val_accuracy: 0.5220
Epoch 3/100
110/110 [==============================] - 7s 60ms/step - loss: 1.1936 - accuracy

Epoch 6/100
110/110 [==============================] - 7s 62ms/step - loss: 0.2674 - accuracy: 0.9113 - val_loss: 0.6068 - val_accuracy: 0.8060
Epoch 7/100
110/110 [==============================] - 7s 62ms/step - loss: 0.2851 - accuracy: 0.9007 - val_loss: 0.3536 - val_accuracy: 0.8900
Epoch 8/100
110/110 [==============================] - 7s 62ms/step - loss: 0.2393 - accuracy: 0.9139 - val_loss: 0.3946 - val_accuracy: 0.8960
Epoch 9/100
110/110 [==============================] - 7s 62ms/step - loss: 0.2003 - accuracy: 0.9306 - val_loss: 0.4479 - val_accuracy: 0.8700
Epoch 10/100
110/110 [==============================] - 7s 62ms/step - loss: 0.1575 - accuracy: 0.9452 - val_loss: 0.4717 - val_accuracy: 0.8940
Epoch 11/100
110/110 [==============================] - 7s 62ms/step - loss: 0.1561 - accuracy: 0.9423 - val_loss: 0.4414 - val_accuracy: 0.8900
Epoch 12/100
110/110 [==============================] - 7s 65ms/step - loss: 0.1084 - accuracy: 0.9629 - val_loss: 0.4624 - val_accura

110/110 [==============================] - 7s 60ms/step - loss: 0.0153 - accuracy: 0.9965 - val_loss: 0.5750 - val_accuracy: 0.9020
Epoch 63/100
110/110 [==============================] - 7s 60ms/step - loss: 0.0494 - accuracy: 0.9851 - val_loss: 0.6299 - val_accuracy: 0.9100
Epoch 64/100
110/110 [==============================] - 6s 59ms/step - loss: 0.0159 - accuracy: 0.9950 - val_loss: 0.6744 - val_accuracy: 0.9140
Epoch 65/100
110/110 [==============================] - 7s 60ms/step - loss: 0.0083 - accuracy: 0.9983 - val_loss: 0.8550 - val_accuracy: 0.9140
Epoch 66/100
110/110 [==============================] - 7s 61ms/step - loss: 0.0109 - accuracy: 0.9968 - val_loss: 0.8194 - val_accuracy: 0.9040
Epoch 67/100
110/110 [==============================] - 7s 60ms/step - loss: 0.0093 - accuracy: 0.9972 - val_loss: 0.9543 - val_accuracy: 0.8980
Epoch 68/100
110/110 [==============================] - 7s 59ms/step - loss: 0.0129 - accuracy: 0.9970 - val_loss: 0.6462 - val_accuracy: 0.908

Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
Test Accuracy: 91.39999747276306
Done!

  Activation Filters    Acc
0       relu       1  0.936
1       relu       2  0.910
2       relu       3  0.912
3       relu       4  0.920
4       relu       5  0.914

-------------------------------------------
Training 6: relu activation, 6 kernel size.
-------------------------------------------
Epoch 1/100
110/110 [==============================] - 11s 67ms/step - loss: 1.8668 - accuracy: 0.2596 - val_loss: 1.3181 - val_accuracy: 0.4660
Epoch 2/100
110/110 [==============================] - 6s 58ms/step - loss: 1.1485 - accuracy: 0.5459 - val_loss: 0.7227 - val_accuracy: 0.7620
Epoch 3/100
110/110 [==============================] - 7s 60ms/step - loss: 0.7327 - accuracy: 0.7379 - val_loss: 0.5461 - val_accuracy: 0.8020
Epoch 4/100
110/110 [==============================] - 7s 60ms/step - loss: 0.5719 - accuracy: 0.7999 - val_loss: 0.4055 - val_accuracy: 0.862

## Summary

In [32]:
record2.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
0,relu,1,0.936
3,relu,4,0.920
5,relu,6,0.918
4,relu,5,0.914
2,relu,3,0.912
1,relu,2,0.910


In [33]:
report = record2.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## TCN Model

In [35]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(6, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [36]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 100, 300)     300000      input_21[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_20 (SpatialDr (None, 100, 300)     0           embedding_20[0][0]               
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_20[0][0]       
___________________________________________________________________________________________

## Train and Test the Model

In [37]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [41]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record3 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):

            model.fit(Xtrain, train_y, batch_size=50, epochs=200, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/200
110/110 [==============================] - 16s 105ms/step - loss: 1.7516 - accuracy: 0.2901 - val_loss: 0.9225 - val_accuracy: 0.6700
Epoch 2/200
110/110 [==============================] - 11s 104ms/step - loss: 0.8424 - accuracy: 0.6706 - val_loss: 0.6787 - val_accuracy: 0.7800
Epoch 3/200
110/110 [==============================] - 11s 100ms/step - loss: 0.4505 - accuracy: 0.8515 - val_loss: 0.3809 - val_accuracy: 0.8900
Epoch 4/200
110/110 [==============================] - 12s 109ms/step - loss: 0.2638 - accuracy: 0.9109 - val_loss: 0.3444 - val_accuracy: 0.8740
Epoch 5/200
110/110 [==============================] - 11s 104ms/step - loss: 0.1397 - accuracy: 0.9541 - val_loss: 0.6288 - val_accuracy: 0.8160
Epoch 6/200
110/110 [==============================] - 10s 95ms/step - loss: 0.1158 - accuracy: 0.9593 - val_loss: 0.4092 - val_accuracy: 

110/110 [==============================] - 10s 94ms/step - loss: 0.0134 - accuracy: 0.9966 - val_loss: 1.1205 - val_accuracy: 0.8840
Epoch 28/200
110/110 [==============================] - 10s 94ms/step - loss: 0.0145 - accuracy: 0.9948 - val_loss: 0.9473 - val_accuracy: 0.8080
Epoch 29/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0247 - accuracy: 0.9952 - val_loss: 0.7900 - val_accuracy: 0.8040
Epoch 30/200
110/110 [==============================] - 10s 95ms/step - loss: 0.0220 - accuracy: 0.9934 - val_loss: 0.7014 - val_accuracy: 0.8800
Epoch 31/200
110/110 [==============================] - 10s 94ms/step - loss: 0.0582 - accuracy: 0.9847 - val_loss: 0.7420 - val_accuracy: 0.8920
Epoch 32/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0124 - accuracy: 0.9964 - val_loss: 1.1498 - val_accuracy: 0.8740
Epoch 33/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 1.3990 - val_accuracy

110/110 [==============================] - 11s 96ms/step - loss: 1.6574 - accuracy: 0.2416 - val_loss: 1.6894 - val_accuracy: 0.1880
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 18.799999356269836
The model suffered from local minimum. Retrain the model!
Epoch 1/200
110/110 [==============================] - 19s 126ms/step - loss: 2.0384 - accuracy: 0.1778 - val_loss: 1.6052 - val_accuracy: 0.4180
Epoch 2/200
110/110 [==============================] - 13s 114ms/step - loss: 1.3653 - accuracy: 0.4320 - val_loss: 0.6360 - val_accuracy: 0.7820
Epoch 3/200
110/110 [==============================] - 11s 100ms/step - loss: 0.6451 - accuracy: 0.7603 - val_loss: 0.5350 - val_accuracy: 0.8220
Epoch 4/200
110/110 [==============================] - 11s 99ms/step - loss: 0.3969 - accuracy: 0.8350 - val_loss: 0.4327 - val_accuracy: 0.8800
Epoch 5/200
110/110 [==============================] - 11s 99ms/step - loss: 0.2960 - accuracy: 0.8716 - val

Epoch 55/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0295 - accuracy: 0.9941 - val_loss: 1.1467 - val_accuracy: 0.8460
Epoch 56/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0274 - accuracy: 0.9946 - val_loss: 0.9191 - val_accuracy: 0.8700
Epoch 57/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0024 - accuracy: 0.9996 - val_loss: 1.0247 - val_accuracy: 0.8720
Epoch 58/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 1.0522 - val_accuracy: 0.8540
Epoch 59/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0151 - accuracy: 0.9964 - val_loss: 1.1068 - val_accuracy: 0.8820
Epoch 60/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 1.1741 - val_accuracy: 0.8800
Restoring model weights from the end of the best epoch.
Epoch 00060: early stopping
Test Accuracy: 90.79999923706055
Done!



110/110 [==============================] - 10s 92ms/step - loss: 0.0646 - accuracy: 0.9743 - val_loss: 0.6265 - val_accuracy: 0.9040
Epoch 13/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0650 - accuracy: 0.9817 - val_loss: 0.6115 - val_accuracy: 0.9040
Epoch 14/200
110/110 [==============================] - 10s 91ms/step - loss: 0.0539 - accuracy: 0.9836 - val_loss: 0.6074 - val_accuracy: 0.8980
Epoch 15/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0282 - accuracy: 0.9898 - val_loss: 0.6731 - val_accuracy: 0.8920
Epoch 16/200
110/110 [==============================] - 10s 93ms/step - loss: 0.0424 - accuracy: 0.9862 - val_loss: 0.6615 - val_accuracy: 0.8940
Epoch 17/200
110/110 [==============================] - 10s 92ms/step - loss: 0.0342 - accuracy: 0.9877 - val_loss: 0.9204 - val_accuracy: 0.8760
Epoch 18/200
110/110 [==============================] - 10s 94ms/step - loss: 0.0700 - accuracy: 0.9793 - val_loss: 0.5299 - val_accuracy

## Summary

In [42]:
record3.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
5,relu,6,0.918
4,relu,5,0.916
0,relu,1,0.908
3,relu,4,0.908
1,relu,2,0.906
2,relu,3,0.898


In [43]:
report = record3.sort_values(by='Acc', ascending=False)
report = report.to_excel('TCN_TREC_3.xlsx', sheet_name='dynamic')